In [24]:
%%file sumArraysOnGPU.cu

#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <stdio.h>
#include <cuda_runtime.h>
#include <sys/time.h>

double cpuSecond(){
    struct timeval tp;
    gettimeofday(&tp, NULL);
    return ((double)tp.tv_sec + (double)tp.tv_usec*1.e-6);
}

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(1);                                                               \
    }                                                                          \
}


__global__ void sumArraysOnDevice(float *A, float *B, float *C, const int N){
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) C[idx] = A[idx] + B[idx];

}


void initialData(float *ip, int size){
    // generate different seed for random number 
    time_t t;
    srand((unsigned int) time (&t));
    
    for (int i=0; i<size; i++){
        ip[i] = (float)(rand() & 0xFF) / 10.0f;
    }
}


void sumArraysOnHost(float *A, float *B, float *C, const int N){
    for (int idx=0; idx<N; idx++){
        C[idx] = A[idx] + B[idx];
    }
}



void checkResult(float *hostRef, float *gpuRef, const int N){
    double epsilon = 1.0E-8;
    int match = 1;
    for (int i = 0; i < N; i++){
        if (abs(hostRef[i] - gpuRef[i]) > epsilon){
            match = 0;
            printf("Arrays do not match!\n");
            printf("host %5.2f gpu %5.2f at current %d\n",
                   hostRef[i], gpuRef[i], i);
            break;
        }
    }
    if (match) printf("Arrays match. \n\n");
}


int main(int argc, char **argv){
    
    printf("%s Starting...\n", argv[0]);
    
    // malloc host memory
    int nElem = 1 <<24;
    size_t nBytes = nElem * sizeof(float);
    
    
    // initialize data at host side
    float *h_A, *h_B, *hostRef, *gpuRef;
    h_A = (float *)malloc(nBytes);
    h_B = (float *)malloc(nBytes);
    hostRef = (float *)malloc(nBytes);
    gpuRef = (float *)malloc(nBytes);
    
    // initialize data at host side
    initialData(h_A, nElem);
    initialData(h_B, nElem);
    
    memset(hostRef, 0, nBytes);
    memset(gpuRef, 0, nBytes);
    
    // malloc device global memory 
    float *d_A, *d_B, *d_C;
    cudaMalloc((float**)&d_A, nBytes);
    cudaMalloc((float**)&d_B, nBytes);
    cudaMalloc((float**)&d_C, nBytes);
    
    // Use cudaMemcpy to transfer the data from the host memory to the GPU global memory with the
    // parameter cudaMemcpyHostToDevice specifying the transfer direction.
    
    CHECK(cudaMemcpy(d_A, h_A, nBytes, cudaMemcpyHostToDevice));
    CHECK(cudaMemcpy(d_B, h_B, nBytes, cudaMemcpyHostToDevice));
    
    // invoke kernel at host side
    int iLen = 128;
    dim3 block(iLen);
    dim3 grid((nElem+block.x-1)/block.x);
    
    double iStart = cpuSecond();
    sumArraysOnDevice<<<grid, block>>>(d_A, d_B, d_C, nElem);
    CHECK(cudaDeviceSynchronize());
    double iElaps = cpuSecond() - iStart;
    printf("sumArraysOnGPU <<<%d,%d>>> Time elapsed %f sec\n", grid.x, block.x, iElaps);
    //printf("Execution configuration <<<%d, %d>>>\n", grid.x, block.x);
    
    // copy kernel result back to host side 
    cudaMemcpy(gpuRef, d_C, nBytes, cudaMemcpyDeviceToHost);
    
    // add vector at host side for result checks
    sumArraysOnHost(h_A, h_B, hostRef, nElem);
    
    for (int i=0; i<10; i++){
         printf("%f + %f = %f \n", h_A[i], h_B[i], hostRef[i]);

    }
    
    // check device results
    checkResult(hostRef, gpuRef, nElem);
    
    free(h_A);
    free(h_B);
    free(hostRef);
    free(gpuRef);
    
    // use cudaFree to release the memory used on the GPU
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    cudaDeviceReset();
    
    return (0);
}


Overwriting sumArraysOnGPU.cu


In [26]:
%%bash
nvcc sumArraysOnGPU.cu -o addvector
./addvector

./addvector Starting...
sumArraysOnGPU <<<131072,128>>> Time elapsed 0.016467 sec
2.800000 + 2.800000 = 5.600000 
10.000000 + 10.000000 = 20.000000 
2.600000 + 2.600000 = 5.200000 
22.299999 + 22.299999 = 44.599998 
11.000000 + 11.000000 = 22.000000 
9.900000 + 9.900000 = 19.799999 
14.600000 + 14.600000 = 29.200001 
22.299999 + 22.299999 = 44.599998 
21.100000 + 21.100000 = 42.200001 
8.600000 + 8.600000 = 17.200001 
Arrays match. 



## Timing with nvprof

In [27]:
!nvprof --unified-memory-profiling off ./addvector

./addvector Starting...
==19639== NVPROF is profiling process 19639, command: ./addvector
sumArraysOnGPU <<<131072,128>>> Time elapsed 0.014515 sec
24.600000 + 24.600000 = 49.200001 
11.400000 + 11.400000 = 22.799999 
9.800000 + 9.800000 = 19.600000 
15.000000 + 15.000000 = 30.000000 
0.800000 + 0.800000 = 1.600000 
22.700001 + 22.700001 = 45.400002 
8.800000 + 8.800000 = 17.600000 
17.700001 + 17.700001 = 35.400002 
5.100000 + 5.100000 = 10.200000 
3.800000 + 3.800000 = 7.600000 
Arrays match. 

==19639== Profiling application: ./addvector
==19639== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   61.59%  86.326ms         2  43.163ms  43.142ms  43.184ms  [CUDA memcpy HtoD]
                   28.89%  40.487ms         1  40.487ms  40.487ms  40.487ms  [CUDA memcpy DtoH]
                    9.52%  13.347ms         1  13.347ms  13.347ms  13.347ms  sumArraysOnDevice(float*, float*, float*, int)
      API calls:   40.42%  

In [28]:
!nvprof --help

Usage: nvprof [options] [application] [application-arguments]
Options:
       --aggregate-mode <on|off>
                        Turn on/off aggregate mode for events and metrics specified
                        by subsequent "--events" and "--metrics" options. Those
                        event/metric values will be collected for each domain instance,
                        instead of the whole device. Allowed values:
                        	on - turn on aggregate mode (default)
                        	off - turn off aggregate mode

       --analysis-metrics
                        Collect profiling data that can be imported to Visual Profiler's
                        "analysis" mode. Note: Use "--export-profile" to specify
                        an export file.

       --annotate-mpi <off|openmpi|mpich>
                        Automatically annotate MPI calls with NVTX markers. Specify
                        the MPI implementation installed on your machine. Currently,
        